# Sample code how to use `logllm` package

## Your machine learning code is here

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()

X = iris.data[iris.target != 2] 
y = iris.target[iris.target != 2]  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


## Start logging with `logllm`

In [2]:
from logllm import log_llm

notebook_path = "svc-sample.ipynb" # Here is target file to log
project_name = "logllm-sample" # project name for wandb

log_llm(notebook_path,project_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: baseconnect-wandb (base-connect-wandb). Use `wandb login --relogin` to force relogin


{
    "method": "SVC",
    "dataset": "Iris",
    "task": "classification",
    "is_advanced_method": false,
    "is_latest_method": "",
    "accuracy": 1.00,
    "kernel": "linear",
    "test_size": 0.2,
    "random_state": 42,
    "condition_as_natural_langauge": ["Using linear kernel on SVC model.", "Excluding class 2 from Iris dataset.", "Splitting data into 80% training and 20% testing."],
    "advice_to_improve_acc": ["Confirm dataset consistency.", "Consider cross-validation for validation.", "Experiment with other kernels such as RBF for potential improvement."]
}
Response from OpenAI logged to W&B.


## Expected condition with GPT4 from the script

```Python
{
    "method": "SVC",
    "dataset": "Iris",
    "task": "classification",
    "is_advanced_method": false,
    "is_latest_method": "",
    "accuracy": 1.00,
    "kernel": "linear",
    "test_size": 0.2,
    "random_state": 42,
    "condition_as_natural_langauge": ["Using linear kernel on SVC model.", "Excluding class 2 from Iris dataset.", "Splitting data into 80% training and 20% testing."],
    "advice_to_improve_acc": ["Confirm dataset consistency.", "Consider cross-validation for validation."]
}

```